# <font color="red">Lecture 6</font>

+-------------------------------------Hybrid Search RAG-------------------------------------+

Weaviate - Vector DB - Cloud version

LLM - HuggingFace

+---------------------------------------------------------------------------------------------------------+

<font color="red">Weaviate Cluster在實做完後以被 **delete**，`WEAVIATE_CLUSTER`與`WEAVIATE_API_KEY` 都需被重新產生


In [1]:
!nvidia-smi

Sat Jul 20 10:32:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install weaviate-client --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
go

In [3]:
!pip install langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.7 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.7 MB/s eta 0:00:00


In [5]:
import weaviate

<font color="red">使用的Client library : [Weaviate Python Client v3](https://weaviate.io/developers/weaviate/client-libraries/python/python_v3)

<font color="red">先在 https://weaviate.io/ 建立cluster ，命名為 `hybrid-rag`

In [6]:
WEAVIATE_CLUSTER = "https://...weaviate.cloud"
WEAVIATE_API_KEY = "..."

In [7]:
HF_TOKEN = "..."

In [8]:
import os

In [9]:
client = weaviate.Client(
    url = WEAVIATE_CLUSTER,
    auth_client_secret= weaviate.AuthApiKey(WEAVIATE_API_KEY),
    additional_headers={
         "X-HuggingFace-Api-Key": HF_TOKEN
    },
)

In [10]:
client.is_ready()
# assert client.is_ready() ## client連結成功，不會有異常，client連結不成功，則會拋出異常

True

In [10]:
client.schema.get() ## 新建立的 cluster ，schema 是空的，cluster 若非新建立的，會顯示已有的 schema

{'classes': []}

<font color="red">建立 `schema` ，參考 https://weaviate.io/developers/weaviate/starter-guides/schema#schema-an-introduction

In [9]:
schema = {
  "classes": [
    {
      "class": "RAG",
      "description": "Documents for RAG",
      "vectorizer": "text2vec-huggingface",
      "moduleConfig": {"text2vec-huggingface": {"model": "sentence-transformers/all-MiniLM-L6-v2", "type": "text"}},
      "properties": [
        {
          "dataType": ["text"],
          "description": "The content of the paragraph",
          "moduleConfig": {
            "text2vec-huggingface": {
              "skip": False,
              "vectorizePropertyName": False,
            }
          },
          "name": "content",
        },
      ],
    },
  ]
}
## schema 中，sentence-transformers 就是指定用來處理 embedding 的工作，之後會自動執行，無須再手動處理

In [13]:
client.schema.create(schema)

<font color="red">注意`client.schema.create(schema)`與`client.schema.create_class(schema_class)`的區別:
</font>
* `.create(schema)`:schema 是一個 single class
* `.create_class(schema_class`:schema_class 是一個 list of classes，可以包含多個 class.

In [26]:
client.schema.get()

{'classes': [{'class': 'RAG',
   'description': 'Documents for RAG',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the paragraph',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-huggingface': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Fri Jul 19 10:19:54 2024",
     'indexFilterable': True,
     'indexSearchabl

In [11]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

In [12]:
## 注意 Weaiate Cluster 需要正常啟動
retriever = WeaviateHybridSearchRetriever(
    alpha = 0.5,
    client = client,
    index_name = "RAG",
    text_key = "content",
    attributes= [],
    create_schema_if_missing=True ## if schema is missing, it will automatically create for you.
)

<font color="red">[WeaviateHybridSearchRetriever說明](https://api.python.langchain.com/en/latest/retrievers/langchain_community.retrievers.weaviate_hybrid_search.WeaviateHybridSearchRetriever.html)</font>

In [13]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [14]:
!pip install bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.4 MB/s eta 0:00:00


In [15]:
!pip install accelerate --quiet

In [16]:
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

In [17]:
# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

In [18]:
# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [19]:
tokenizer = initialize_tokenizer(model_name)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [20]:
model = load_quantized_model(model_name)

Unused kwargs: ['low_cpu_mem_usage']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [21]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    #max_length=2048,
    do_sample=True,
    top_k=5,
    max_new_tokens=100,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [22]:
llm = HuggingFacePipeline(pipeline=pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [23]:
# doc_path = "/content/stats.pdf" # Lecture
doc_path = "/content/stats1.pdf" # 實際操作

In [24]:
!pip install pypdf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 5.9 MB/s eta 0:00:00


In [25]:
from langchain_community.document_loaders import PyPDFLoader

In [26]:
loader = PyPDFLoader(doc_path)

In [27]:
docs = loader.load()

## <font color="red">以下是 Lecture 的結果

<font color="red">但沒有原課程 stats.pdf ，所以用自訂的 stats1.pdf 當成輸入，其結果與過程放在下一個 section

In [ ]:
docs ## 包含三個 Document （'page':0 到 'page':2）

[Document(page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis\ninformation\ncan\nbe\nuseful\nfor\nunderstanding

In [ ]:
docs[0]

Document(page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis\ninformation\ncan\nbe\nuseful\nfor\nunderstanding\

In [ ]:
retriever.add_documents(docs) ## 功能 Upload documents to Weaviate. 已經做過，不需再執行

['013a8494-dd19-4ca5-92b7-e64ce8ecbc31',
 '9350abc8-9806-4acc-815b-2686bdee70a2',
 '46f07205-b0f6-42b5-b59d-7cffa96b4f66']

In [ ]:
print(retriever.invoke("what is correlation?")[0].page_content)
## 功能 Invoke the retriever to get relevant documents.
## 回傳是 List[Document]
## 此 [0] 是回傳List中第一個Document，也是與問題關聯性最大的

Correlation
is
used
in
a
wide
variety
of
fields
including
1.
Business:
Correlation
can
be
used
to
identify
relationships
between
different
business
variables,
such
as
sales,
advertising
spending,
and
customer
satisfaction.
This
information
can
be
used
to
make
better
business
decisions,
such
as
how
to
allocate
marketing
resources.
2.
Finance:
Correlation
can
be
used
to
identify
relationships
between
different
financial
assets,
such
as
stocks,
bonds,
and
commodities.
This
information
can
be
used
to
build
diversified
portfolios
that
reduce
risk.
3.
Medicine:
Correlation
can
be
used
to
identify
relationships
between
different
medical
variables,
such
as
risk
factors
for
diseases
and
the
effectiveness
of
treatments.
This
information
can
be
used
to
improve
the
prevention,
diagnosis,
and
treatment
of
diseases.
4.
Psychology:
Correlation
can
be
used
to
identify
relationships
between
different
psychological
variables,
such
as
personality
traits,
cognitive
abilities,
and
mental
disorders.
This
in

In [ ]:
retriever.invoke(
    "what is correlation?",
    score=True
)

[Document(page_content='Correlation\nis\nused\nin\na\nwide\nvariety\nof\nfields\nincluding\n1.\nBusiness:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nbusiness\nvariables,\nsuch\nas\nsales,\nadvertising\nspending,\nand\ncustomer\nsatisfaction.\nThis\ninformation\ncan\nbe\nused\nto\nmake\nbetter\nbusiness\ndecisions,\nsuch\nas\nhow\nto\nallocate\nmarketing\nresources.\n2.\nFinance:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nfinancial\nassets,\nsuch\nas\nstocks,\nbonds,\nand\ncommodities.\nThis\ninformation\ncan\nbe\nused\nto\nbuild\ndiversified\nportfolios\nthat\nreduce\nrisk.\n3.\nMedicine:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nmedical\nvariables,\nsuch\nas\nrisk\nfactors\nfor\ndiseases\nand\nthe\neffectiveness\nof\ntreatments.\nThis\ninformation\ncan\nbe\nused\nto\nimprove\nthe\nprevention,\ndiagnosis,\nand\ntreatment\nof\ndiseases.\n4.\nPsychology:\nCorrelation\ncan\nbe\nused\n

In [ ]:
from langchain.chains import RetrievalQA

In [3]:
from langchain_core.prompts import ChatPromptTemplate

In [15]:
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)

In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the given context to answer the question. If you don't know the answer, say you don't know. Use three sentence maximum and keep the answer concise. Context: {context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])

<font color="red">建立 `ChatPromptTemplate` 另法 </font>

```python
from langchain.prompts import (   
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template=(
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{query}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
```

`chat_prompt` 的結果與 `prompt` 一模一樣

In [19]:
## Entire prompt template 是：
from langchain.prompts import PromptTemplate
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
hybrid_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,)

In [ ]:
results1 = hybrid_chain.invoke("what is Correlation?")
print(results1)

{'query': 'what is Correlation?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nCorrelation\nis\nused\nin\na\nwide\nvariety\nof\nfields\nincluding\n1.\nBusiness:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nbusiness\nvariables,\nsuch\nas\nsales,\nadvertising\nspending,\nand\ncustomer\nsatisfaction.\nThis\ninformation\ncan\nbe\nused\nto\nmake\nbetter\nbusiness\ndecisions,\nsuch\nas\nhow\nto\nallocate\nmarketing\nresources.\n2.\nFinance:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nfinancial\nassets,\nsuch\nas\nstocks,\nbonds,\nand\ncommodities.\nThis\ninformation\ncan\nbe\nused\nto\nbuild\ndiversified\nportfolios\nthat\nreduce\nrisk.\n3.\nMedicine:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nmedical\nvariables,\nsuch\nas\nrisk\nfactors\nfor\ndiseases\nand\nthe

In [ ]:
print(results1['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Correlation
is
used
in
a
wide
variety
of
fields
including
1.
Business:
Correlation
can
be
used
to
identify
relationships
between
different
business
variables,
such
as
sales,
advertising
spending,
and
customer
satisfaction.
This
information
can
be
used
to
make
better
business
decisions,
such
as
how
to
allocate
marketing
resources.
2.
Finance:
Correlation
can
be
used
to
identify
relationships
between
different
financial
assets,
such
as
stocks,
bonds,
and
commodities.
This
information
can
be
used
to
build
diversified
portfolios
that
reduce
risk.
3.
Medicine:
Correlation
can
be
used
to
identify
relationships
between
different
medical
variables,
such
as
risk
factors
for
diseases
and
the
effectiveness
of
treatments.
This
information
can
be
used
to
improve
the
prevention,
diagnosis,
and
treatment
of
diseases.
4.
Psychology:
Correla

In [ ]:
# query的方式也可以是:
query="What is correlation?"
response = hybrid_chain.invoke({"query":query})

In [ ]:
response

{'query': 'What is correlation?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nCorrelation\nis\nused\nin\na\nwide\nvariety\nof\nfields\nincluding\n1.\nBusiness:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nbusiness\nvariables,\nsuch\nas\nsales,\nadvertising\nspending,\nand\ncustomer\nsatisfaction.\nThis\ninformation\ncan\nbe\nused\nto\nmake\nbetter\nbusiness\ndecisions,\nsuch\nas\nhow\nto\nallocate\nmarketing\nresources.\n2.\nFinance:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nfinancial\nassets,\nsuch\nas\nstocks,\nbonds,\nand\ncommodities.\nThis\ninformation\ncan\nbe\nused\nto\nbuild\ndiversified\nportfolios\nthat\nreduce\nrisk.\n3.\nMedicine:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nmedical\nvariables,\nsuch\nas\nrisk\nfactors\nfor\ndiseases\nand\nth

In [ ]:
## you can pass another way so if you want to also pass it as a 'run pass through'
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
# Set up the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    prompt |
    llm
)

In [ ]:
query="what is correlation?"

In [ ]:
response=rag_chain.invoke("what is correlation?")

In [ ]:
print(response)


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
[Document(page_content='Correlation\nis\nused\nin\na\nwide\nvariety\nof\nfields\nincluding\n1.\nBusiness:\nCorrelation\ncan\nbe\nused\nto\nidentify\nrelationships\nbetween\ndifferent\nbusiness\nvariables,\nsuch\nas\nsales,\nadvertising\nspending,\nand\ncustomer\nsatisfaction.\nThis\ninformation\ncan\nbe\nused\nto\nmake\nbetter\nbusiness\ndecisions,\nsuch\nas\nhow\nto\nallocate\nmarketing\nresources.\n2.\nFinance:\

## <font color="red">以下是 實際運行 的結果

<font color="red">用自訂的 stats1.pdf 當成輸入

In [28]:
docs ## 包含四個 Document （'page':0 到 'page':3）

[Document(metadata={'source': '/content/stats1.pdf', 'page': 0}, page_content='Dr. Alok Kumar  \nDept. of Economics  \nM.A. 1st Sem  \nCORRELATION: TYPES  \nBefore going through the types of correlation let’s learn about scatter \ndiagram.  \n \nScatter Diagram  \nA scatter diagram is a diagram that shows the values of two variables X \nand Y , along with the way in which these two variables relate to each \nother. The values of variable X are given along the horizontal axis, with \nthe values of the variable Y given on the vertical axis. For purposes of \ndrawing a scatter diagram, and determining the corre lation coefficient, it \ndoes not matter which of the two variables is the X variable, and which \nis Y.  \nLater, when the regression model is used, one of the variables is defined \nas an independent variable, and the other is defined as a dependent \nvariable. In regression, the independent variable X is considered to have \nsome effect or influence on the dependent variable Y. 

In [31]:
docs[0]

Document(metadata={'source': '/content/stats1.pdf', 'page': 0}, page_content='Dr. Alok Kumar  \nDept. of Economics  \nM.A. 1st Sem  \nCORRELATION: TYPES  \nBefore going through the types of correlation let’s learn about scatter \ndiagram.  \n \nScatter Diagram  \nA scatter diagram is a diagram that shows the values of two variables X \nand Y , along with the way in which these two variables relate to each \nother. The values of variable X are given along the horizontal axis, with \nthe values of the variable Y given on the vertical axis. For purposes of \ndrawing a scatter diagram, and determining the corre lation coefficient, it \ndoes not matter which of the two variables is the X variable, and which \nis Y.  \nLater, when the regression model is used, one of the variables is defined \nas an independent variable, and the other is defined as a dependent \nvariable. In regression, the independent variable X is considered to have \nsome effect or influence on the dependent variable Y. C

In [32]:
retriever.add_documents(docs) ## 功能 Upload documents to Weaviate. 已經做過，不需再執行

['a1a9b1d0-bb9a-47ef-b9ec-d91008d9ad60',
 'f1569306-9cd1-4930-8ea9-1b1799a1363c',
 '3f5958ee-8bb2-4464-b20f-7cbe7d60c8fe',
 '77af706b-7ae0-42ec-a0ba-bc1e5f1d2352']

In [29]:
print(retriever.invoke("what is positive and negative Correlation?")[0].page_content)
## 功能 Invoke the retriever to get relevant documents.
## 回傳是 List[Document]
## 此 [0] 是回傳List中第一個Document，也是與問題關聯性最大的

 
1.Positive and Negative Correlation: Whether the correlation between 
the variables is positive or negative depe nds on its direction of change . 
The correlation is positive when both the variables move in the same 
direction , i.e. when one variable increases the other on an average also 
increases and if one variable decreases the other also decreases.The 
correlation i s said to be negative when both the variables move in the 
opposite direction , i.e. when one variable increases the other decreases 
and vice versa.  
2.Simple, Partial and Multiple Correlation:  Whether the correlation is 
simple, partial or multiple depends on  the number of variables studied . 
The correlation is said to be simple when only two variables  are 
studied.The correlation is either multiple or partial when three or more 
variables are studied. The correlation is said to be Multiple when three 
variables a re studied simultaneously . Such as, if we want to study the 
relationship between the yield of

In [34]:
retriever.invoke(
    "what is positive and negative correlation?",
    score=True
)

[Document(metadata={'_additional': {'explainScore': '\nHybrid (Result Set keyword,bm25) Document 3f5958ee-8bb2-4464-b20f-7cbe7d60c8fe: original score 1.3991178, normalized score: 0.5 - \nHybrid (Result Set vector,hybridVector) Document 3f5958ee-8bb2-4464-b20f-7cbe7d60c8fe: original score 0.66872144, normalized score: 0.5', 'score': '1'}}, page_content=' \n1.Positive and Negative Correlation: Whether the correlation between \nthe variables is positive or negative depe nds on its direction of change . \nThe correlation is positive when both the variables move in the same \ndirection , i.e. when one variable increases the other on an average also \nincreases and if one variable decreases the other also decreases.The \ncorrelation i s said to be negative when both the variables move in the \nopposite direction , i.e. when one variable increases the other decreases \nand vice versa.  \n2.Simple, Partial and Multiple Correlation:  Whether the correlation is \nsimple, partial or multiple depe

In [28]:
from langchain.chains import RetrievalQA

In [29]:
# 似乎沒用
from langchain_core.prompts import ChatPromptTemplate

In [30]:
# 似乎沒用
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

In [46]:
# 似乎沒用 (使用此 prompt ，在下個 section 展示)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)

In [47]:
from langchain.prompts import PromptTemplate
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain # 似乎沒用

In [32]:
hybrid_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,)


In [42]:
## 每次執行，結果可能會不同
results1 = hybrid_chain.invoke("what is positive and negative correlation?")
print(results1)

{'query': 'what is positive and negative correlation?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n \n1.Positive and Negative Correlation: Whether the correlation between \nthe variables is positive or negative depe nds on its direction of change . \nThe correlation is positive when both the variables move in the same \ndirection , i.e. when one variable increases the other on an average also \nincreases and if one variable decreases the other also decreases.The \ncorrelation i s said to be negative when both the variables move in the \nopposite direction , i.e. when one variable increases the other decreases \nand vice versa.  \n2.Simple, Partial and Multiple Correlation:  Whether the correlation is \nsimple, partial or multiple depends on  the number of variables studied . \nThe correlation is said to be simple when only two variables  are \nstudied.Th

In [43]:
print(results1['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

 
1.Positive and Negative Correlation: Whether the correlation between 
the variables is positive or negative depe nds on its direction of change . 
The correlation is positive when both the variables move in the same 
direction , i.e. when one variable increases the other on an average also 
increases and if one variable decreases the other also decreases.The 
correlation i s said to be negative when both the variables move in the 
opposite direction , i.e. when one variable increases the other decreases 
and vice versa.  
2.Simple, Partial and Multiple Correlation:  Whether the correlation is 
simple, partial or multiple depends on  the number of variables studied . 
The correlation is said to be simple when only two variables  are 
studied.The correlation is either multiple or partial when three or more 
variables are stu

In [44]:
query="what is positive and negative correlation?"
response = hybrid_chain.invoke({"query":query})

In [45]:
response

{'query': 'what is positive and negative correlation?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n \n1.Positive and Negative Correlation: Whether the correlation between \nthe variables is positive or negative depe nds on its direction of change . \nThe correlation is positive when both the variables move in the same \ndirection , i.e. when one variable increases the other on an average also \nincreases and if one variable decreases the other also decreases.The \ncorrelation i s said to be negative when both the variables move in the \nopposite direction , i.e. when one variable increases the other decreases \nand vice versa.  \n2.Simple, Partial and Multiple Correlation:  Whether the correlation is \nsimple, partial or multiple depends on  the number of variables studied . \nThe correlation is said to be simple when only two variables  are \nstudied.T

In [39]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough # RunnableParallel 似乎沒用到

In [49]:
# Set up the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    prompt |
    llm
)

In [44]:
query="what is positive and negative correlation?"

In [50]:
response=rag_chain.invoke("what is positive and negative correlation?")

In [50]:
print(response)


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
[Document(page_content=' \n1.Positive and Negative Correlation: Whether the correlation between \nthe variables is positive or negative depe nds on its direction of change . \nThe correlation is positive when both the variables move in the same \ndirection , i.e. when one variable increases the other on an average also \nincreases and if one variable decreases the other also decreases.The \ncorrelation i s said to

In [52]:
print(response)


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
[Document(page_content=' \n1.Positive and Negative Correlation: Whether the correlation between \nthe variables is positive or negative depe nds on its direction of change . \nThe correlation is positive when both the variables move in the same \ndirection , i.e. when one variable increases the other on an average also \nincreases and if one variable decreases the other also decreases.The \ncorrelation i s said to

## <font color="red">(補充)ChatPromptTemplate 使用實例

In [32]:
from langchain.chains import ConversationalRetrievalChain

In [34]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": prompt} ## prompt 是來自上一個 section 的 ChatPromptTemplate.from_messages
)

In [40]:
# query = input('\nQ: ')
query = "what is positive and negative correlation?"
result = qa({"question": query,'query':query, 'chat_history': []})
# 也可以是 qa.invoke({"question": query,"query": query, 'chat_history': []})
# 不知原因，"question"與"query"都必需要，缺任一，都會出`missing some input key...`

In [42]:
print(result['answer'])

System: Use the given context to answer the question. If you don't know the answer, say you don't know. Use three sentence maximum and keep the answer concise. Context:  
1.Positive and Negative Correlation: Whether the correlation between 
the variables is positive or negative depe nds on its direction of change . 
The correlation is positive when both the variables move in the same 
direction , i.e. when one variable increases the other on an average also 
increases and if one variable decreases the other also decreases.The 
correlation i s said to be negative when both the variables move in the 
opposite direction , i.e. when one variable increases the other decreases 
and vice versa.  
2.Simple, Partial and Multiple Correlation:  Whether the correlation is 
simple, partial or multiple depends on  the number of variables studied . 
The correlation is said to be simple when only two variables  are 
studied.The correlation is either multiple or partial when three or more 
variables ar